In [8]:
import os
if os.getcwd() == '/home/user/code':
    os.chdir('/home/user/code/nlp2024_ClefTask4SOTA')

In [9]:
from TexSoup import TexSoup
import re

def find_sections(tex):
    return [match.group(1) for match in re.finditer("\\\section\{(.*)\}", tex)]


def extract_content(node, name):
    if node:
        return ((name, node.contents[0]))


def section_split(tex):
    """returns: List of tuples (section_name, section_text)"""
    doc_text = tex
    sections = []
    # parsing with TexSoup
    # TODO: get rid of this and use regex only (brittle)
    try:
        soup = TexSoup(tex, tolerance=1)
        if title:=extract_content(soup.title, "title"):
            sections.append(title)
        if abstract:=extract_content(soup.abstract, "abstract"):
            sections.append(abstract)
        sections.append(("tables", "\n".join([str(node) for node in soup.find_all("table")]))) if soup.find_all("table") else None
    except:
        pass # could not parse tex

    # extract latex sections and corresponding text
    prev_section = "pre"

    for section in find_sections(tex):
        section_text, doc_text = doc_text.split(f"\\section{{{section}}}", 1)
        sections.append((prev_section, section_text))
        prev_section = section
    sections.append((prev_section, doc_text))
    return sections
    # except:
    #     return (("full", tex))
    


<>:5: SyntaxWarning: invalid escape sequence '\s'
<>:5: SyntaxWarning: invalid escape sequence '\s'
C:\Users\urban\AppData\Local\Temp\ipykernel_33512\3670686320.py:5: SyntaxWarning: invalid escape sequence '\s'
  return [match.group(1) for match in re.finditer("\\\section\{(.*)\}", tex)]


In [10]:
import ollama


def pass_to_ollama(prompt, model):
    try:
        res = ollama.generate(model=model, prompt=prompt, options={"temperature": 0})
        return res["response"]
    except Exception as ex:
        print(ex)
        return f"ollama error: {ex}"


In [11]:
# pass_to_ollama("test", "llama3:8b")

In [16]:
import json

from src.dataset import UNANSWERABLE


# prompt template
def extract_tdms(tex, few_shot=True):
    if few_shot:
        return f"""If the text reports benchmark leaderboard results, extract the reported Tasks, Datasets, Metrics and corresponding Scores.
        
        Text: {tex}
        
        Return the tasks, datasets, metrics and scores as reported in the text in a JSON array. Do not include precision information in the reported score.
        Here the formating structure of the JSON. Please use exactly this formating in your answer.
        [
            {{"Task": "example Task 1", "Dataset": "example Dataset 1", "Metric": example metric 1", "Score": "score"}}, 
            {{"Task": "example Task 1", "Dataset": "example Dataset 2", "Metric": example metric 2", "Score": "score"}}
        ]
        
        Lets make an example for you: Template-Based Automatic Search of Compact Semantic Segmentation Architectures... One discovered architecture achieves 63.2% mean IoU on CamVid and 67.8% on CityScapes having only 270K parameters... evaluation.
        
        The expected answer of you is:
        [
            {{"Task": "Compact Sementic Segmentation", "Dataset": "CamVid", "Metric": Mean IoU", "Score": "63.2"}}, 
            {{"Task": "Compact Sementic Segmentation", "Dataset": "CityScapes", "Metric": Mean IoU", "Score": "67.8"}}
        ]
        
        """
    else:
        return f"""If the text reports benchmark leaderboard results, extract the reported Tasks, Datasets, Metrics and corresponding Scores.
        
        Text: {tex}
        
        Return the tasks, datasets, metrics and scores as reported in the text in a JSON array. Do not include precision information in the reported score.
        Here the formating structure of the JSON. Please use exactly this formating in your answer.
        [
            {{"Task": "example Task 1", "Dataset": "example Dataset 1", "Metric": example metric 1", "Score": "score"}}, 
            {{"Task": "example Task 1", "Dataset": "example Dataset 2", "Metric": example metric 2", "Score": "score"}}
        ]
        
        """
 

zero_shot_template = lambda tex: extract_tdms(tex, False)
few_shot_template = lambda tex: extract_tdms(tex, True)

def convert_tdms_to_tuple(model_output_parsed):
    tuples = []
    for item in model_output_parsed:
        try:
            t = ((item["Task"], item["Dataset"],item["Metric"],item["Score"]))
            tuples.append(t)
        except:
            # parse error, ignore instance
            pass
    return tuples

def format_tdms(tuples):
    """make unique, format as string"""
    unique = set(tuples)
    dicts = [{"LEADERBOARD": {
        "Task": t,
        "Dataset":d,
        "Metric":m,
        "Score":s
    }} for t,d,m,s in unique]
    return str(dicts)

model = "llama3:8b"

def section_wise_extraction(tex, prompt_template):
    """takes a tex file, returns an annotation"""
    found_tdms = []
    sections = section_split(str(tex))
    for section_name, section_text in sections:
        prompt = prompt_template(section_text)
        response = pass_to_ollama(prompt, model)

        try:
            response = json.loads(response)
            parsed = convert_tdms_to_tuple(response)
            found_tdms= [*found_tdms, *parsed]
        except:
            pass # no tuples found in section

    # print(found_tdms)
    if not found_tdms:
        annotation = UNANSWERABLE # found_tdms are empty -> unanswerable
    else:
        # dedupe and format
        annotation = format_tdms(found_tdms)
    return annotation

In [18]:
from src.dataset import PATH
from src.experiment_runner import run

# Run on Test
df_test = run(lambda tex: section_wise_extraction(tex, zero_shot_template), PATH.TEST, "llama3_8b_zeroshot")
df_test = run(lambda tex: section_wise_extraction(tex, few_shot_template), PATH.TEST, "llama3_8b_fewshot")


  0%|          | 0/789 [00:00<?, ?it/s]

If the text reports benchmark leaderboard results, extract the reported Tasks, Datasets, Metrics and corresponding Scores.
        
        Text: 

\documentclass[twoside]{article}
\usepackage[latin1]{inputenc}
\usepackage[OT1]{fontenc}
\usepackage{actes}

\ifx\WhizzyStart\undefined\relax\else\textheight 15cm\fi

\def\MarginMark{\marginpar[{\hskip -1em\rule[-7.5mm]{1mm}{5mm}}]{{\hskip -0.3em\rule[-7.5mm]{1mm}{5mm}}}}




\long\def\Modif#1{{\red\sf\MarginMark #1}}

\def\NotreTitre{Typer la dé-sérialisation sans sérialiser les types}
\usepackage[english,francais]{babel}
\usepackage{frbib}

\title{\NotreTitre}
\author{Grégoire {\sc Henry}$^1$, Michel {\sc Mauny}$^2$ et Emmanuel {\sc Chailloux}$^3$}
\titlehead{\NotreTitre}\authorhead{Henry, Mauny \& Chailloux}\affiliation{\begin{tabular}{rr} 
    \\ 1:  Laboratoire Preuves, Programmation et Systèmes (PPS),
    \\ CNRS UMR 7126, Université Denis Diderot, Paris
    \\ {\tt Gregoire.Henry@pps.jussieu.fr}
    \\ 2:  ENSTA, 32 Boulevard Victor,